## 4 PhotoDB and SQL (5 points)

Copyright Jens Dittrich, Christian Schön & Joris Nix, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

In [1]:
import duckdb

## Load Data

Create database tables.

In [2]:
duckdb.sql("""
CREATE TABLE persons (
    id INTEGER PRIMARY KEY,
    lastname TEXT,
    firstname TEXT,
    birthday TEXT
);""")

duckdb.sql("""
CREATE TABLE employees (
    personId INTEGER PRIMARY KEY,
    salary INTEGER,
    experience INTEGER,
    FOREIGN KEY(personId) REFERENCES persons(id)
);""")

duckdb.sql("""
CREATE TABLE seniors (
    employeeId INTEGER PRIMARY KEY,
    numGreyHairs INTEGER,
    bonus INTEGER,
    FOREIGN KEY(employeeId) REFERENCES employees(personId)
);""")

duckdb.sql("""
CREATE TABLE salespersons (
    employeeId INTEGER PRIMARY KEY,
    areaOfExpertise TEXT,
    FOREIGN KEY(employeeId) REFERENCES employees(personId)
);""")

duckdb.sql("""
CREATE TABLE photographers (
    employeeId INTEGER PRIMARY KEY,
    FOREIGN KEY(employeeId) REFERENCES employees(personId)
);""")

duckdb.sql("""
CREATE TABLE cameras (
    id INTEGER PRIMARY KEY,
    brand TEXT,
    model TEXT
);""")

duckdb.sql("""
CREATE TABLE photos (
    id INTEGER PRIMARY KEY,
    location TEXT,
    unix_time INTEGER,
    photographerId INTEGER,
    cameraId INTEGER,
    FOREIGN KEY(photographerId) REFERENCES photographers(employeeId),
    FOREIGN KEY(cameraId) REFERENCES cameras(id)
);""")

Import the csv-data into those tables.

In [3]:
duckdb.sql("COPY persons FROM './data/photodb/persons.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql("COPY employees FROM './data/photodb/employees.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql("COPY seniors FROM './data/photodb/seniors.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql("COPY salespersons FROM './data/photodb/salespersons.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql("COPY photographers FROM './data/photodb/photographers.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql("COPY cameras FROM './data/photodb/cameras.csv' (FORMAT CSV, DELIMITER ',');")
duckdb.sql("COPY photos FROM './data/photodb/photos.csv' (FORMAT CSV, DELIMITER ',');")

### Query 1

The bonus of each senior photographer, that has taken more than two photos with a camera of the brand `Nikon`.

In [4]:
duckdb.sql("DROP VIEW IF EXISTS q1;")

duckdb.sql("""
CREATE VIEW q1 AS
    select s.bonus
    from seniors as s
    join employees as e on s.employeeId = e.personId
    join persons as pe on e.personId = pe.id
    join photographers as p on e.personId = p.employeeId
    join photos as ph on p.employeeId = ph.photographerId
    join cameras as c on ph.cameraId = c.id
    where c.brand = 'Nikon'
    group by pe.id, pe.firstname, pe.lastname, s.bonus
    having count(ph.id) > 2
    ;
""")

Printing output for Query 1

In [5]:
duckdb.sql("""
SELECT *
FROM q1""")

┌───────┐
│ bonus │
│ int32 │
├───────┤
│ 38000 │
│ 55000 │
└───────┘

In [6]:
## TEST
## Prepare the necessary table for result comparison and load the data
## You need to execute this cell only once
## Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

duckdb.sql("""
DROP TABLE IF EXISTS q1_reference;
CREATE TABLE q1_reference (
    bonus INTEGER
);""")

## import query results
duckdb.sql("COPY q1_reference FROM './data/photodb/tests/q1.csv' (FORMAT CSV, DELIMITER ',');")


## Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.
## It also does not consider duplicates.

## compare query results
duckdb.sql("""
SELECT *
FROM (SELECT * FROM q1
      EXCEPT
      SELECT * FROM q1_reference)
UNION
SELECT *
FROM (SELECT * FROM q1_reference
      EXCEPT
      SELECT * FROM q1);""")
## We expect an empty result.

┌────────┐
│ bonus  │
│ int32  │
├────────┤
│ 0 rows │
└────────┘

### Query 2

The brands of the cameras that were used by a photographer that earns a higher salary than the average of all employees. Order your output descendingly according to the brand and only output the first 2 tuples.

In [7]:
duckdb.sql("DROP VIEW IF EXISTS q2;")

duckdb.sql("""
CREATE VIEW q2 AS
    select distinct c.brand
    from cameras as c
    join photos as ph on c.id = ph.cameraId
    join photographers as p on ph.photographerId = p.employeeId
    join employees as e on p.employeeId = e.personId
    where e.salary > (
        select avg(e.salary)
        from employees as e
    )
    order by c.brand desc
    limit 2
    ;
""")

Printing Output of Query 2

In [8]:
duckdb.sql("""
SELECT *
FROM q2""")

┌─────────┐
│  brand  │
│ varchar │
├─────────┤
│ Nikon   │
│ Leica   │
└─────────┘

In [9]:
## TEST
## Prepare the necessary table for result comparison and load the data
## You need to execute this cell only once
## Repeated execution will not affect test results, but lead to error messages as you try to import the same data multiple times

duckdb.sql("""
DROP TABLE IF EXISTS q2_reference;
CREATE TABLE q2_reference (
    brand TEXT PRIMARY KEY
);""")

## import query results
duckdb.sql("COPY q2_reference FROM './data/photodb/tests/q2.csv' (FORMAT CSV, DELIMITER ',');")

## Note that this test compares the resulting tuples and does not ensure that your query is semantically correct.
## It also does not consider duplicates.

## compare query results

duckdb.sql("""
SELECT *
FROM (SELECT * FROM q2
      EXCEPT
      SELECT * FROM q2_reference)
UNION
SELECT *
FROM (SELECT * FROM q2_reference
      EXCEPT
      SELECT * FROM q2);""")
## We expect an empty result.

┌─────────┐
│  brand  │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘